# Quantify kinematic bicycle model (in Vicon)

## Fetch data

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import utm
import scipy
import scipy.interpolate
import scipy.signal
from scipy.spatial.transform import Rotation
# from car_dynamics.envs.kinematic_bicycle_model import LF, LR
from car_dynamics.analysis import pos2vel_savgol, calc_delta, interpolate, rotate_point, calc_delta_v
from offroad.utils import load_state
import ipywidgets as widgets
from ipywidgets import interact
from termcolor import colored
%matplotlib inline
%load_ext autoreload
%autoreload 2

LF = .16
LR = .15

In [2]:
log_dir = '/Users/wenlixiao/Dropbox/School/Graduate/LeCAR/safe-learning-control/playground/offroad/' + 'data/data-20240204-225343'
t_list, p_dict, yaw_dict, action_list, _ = load_state(log_dir, [0, 243], orientation_provider="NONE")
obs_np = p_dict['obs']

## Input To Target Error

In [3]:
def plot_input_to_target_error(p_list, yaw_list, action_list, LF, LR, speed_max=8., delta_offset = -1.6, frequency=20, delta_proj=-1.32,st=0,ed=-1,delay=0):
    '''Error between 
        - vel_real and vel_target
        - steer_real and steer_target          
    '''
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    dt = 1./frequency
    print(dt)
    p_smooth, vel, vel_vec = pos2vel_savgol(p_list,window_length=5, delta=dt)
    delta, beta = calc_delta_v(vel_vec, yaw_list, LF, LR)
    # Project velocity
    vel_real = vel[st:ed]
    vel_tar = action_list[st:ed, 0]*speed_max
    delayed_vel_tar = np.zeros_like(vel_tar)
    delayed_vel_tar[delay:] = vel_tar[:len(vel_tar)-delay]

    # Project Steering
    delta_real = delta[st:ed]
    delta_tar = action_list[st:ed, 1]*delta_proj + delta_offset
    delta_tar = delta_tar[:-1]
    delayed_delta_tar = np.zeros_like(delta_tar)
    delayed_delta_tar[delay:] = delta_tar[:len(delta_tar)-delay]
    axs[0].plot(vel_real,label='real')
    axs[0].plot(delayed_vel_tar,label='target')
    axs[0].set_xlabel(f"frequency: {frequency}")

    axs[1].plot(delta_real,label='real')
    axs[1].plot(delayed_delta_tar,label='target')
    axs[0].set_title("velocity")
    axs[1].set_title("steering")
    axs[0].legend()
    axs[1].legend()
    plt.suptitle("Command (vel, steering) to real error (Vicon input) " + log_dir)

In [4]:
f = lambda st=100,ed=-60, frequency=16.6, speed_max=5.2, delta_offset=-.16, delta_proj=.11, delay=4: plot_input_to_target_error(
                                                obs_np[:,:2], obs_np[:,2], action_list, LF, LR, delta_proj=delta_proj,
                                                speed_max=speed_max, delta_offset = delta_offset, frequency=frequency,
                                                st=st,ed=ed,delay=delay)
interact(f, frequency=(1, 100, 1),
            speed_max=(4., 8., 0.1), 
            steer_max=(0.2, 0.8, 0.01),
            delta_offset=(-4, 4, 0.01),
             delta_proj=(-2., 2., 0.01),
             st=(0,1000,1),
             ed=(-1000,0,1),
             delay=(0,10,1),
        )

interactive(children=(IntSlider(value=100, description='st', max=1000), IntSlider(value=-60, description='ed',…

<function __main__.<lambda>(st=100, ed=-60, frequency=16.6, speed_max=5.2, delta_offset=-0.16, delta_proj=0.11, delay=4)>

## Open Loop Trajectory

In [102]:
def plot_open_loop_trajectory(p_list, yaw_list, action_list, LF, LR, speed_max=8., st=0, ed=-1,
                               steer_max=.34,delta_offset = -1.6, frequency=22, delta_proj=-1.32, beta_offset=0.1, vel_type='target vel', 
                              steer_type='target steer', real_yaw=False,delay=0):
    dt = 1./frequency

    obs_vel_list = p_list[:, 3]
    obs_vel_list[obs_vel_list < .0] = .0
    p_list = p_list[:, :2]
    p_smooth, vel, vel_vec = pos2vel_savgol(p_list,window_length=5, delta=dt)
    # delta, beta = calc_delta(p_smooth, yaw_list, LF, LR)
    delta, beta = calc_delta_v(vel_vec, yaw_list, LF, LR)
   
    vel_tar = action_list[:, 0]*speed_max
    delayed_vel_tar = np.zeros_like(vel_tar)
    delayed_vel_tar[delay:] = vel_tar[:len(vel_tar)-delay]

    delta_tar = action_list[:, 1]*delta_proj + delta_offset
    delta_tar = np.arctan2(np.sin(delta_tar), np.cos(delta_tar))
    delayed_delta_tar = np.zeros_like(delta_tar)
    delayed_delta_tar[delay:] = delta_tar[:len(delta_tar)-delay]
    # plt.plot(obs_vel_list, label='obs')
    # plt.plot(vel, label='calc')
    # plt.legend()
    if vel_type == 'target vel':
        vel = delayed_vel_tar
    elif vel_type =='obs vel':
        vel = obs_vel_list

    if steer_type != 'real steer':
        delta = delayed_delta_tar
        
    x = p_list[st, 0]
    y = p_list[st, 1]
    x_list = [x]
    y_list = [y]
    psi = yaw_list[st]
    delta_list = [delta[st]]
    
    # print(state[1:3]-state[0:2])
    for i, (v_t, beta_t_ground, delta_t, yaw_t) in enumerate(zip(vel[st:ed], beta[st:ed], delta[st:ed], yaw_list[st:ed])):
        if real_yaw:
            beta_t = beta_t_ground
        else:
            beta_t = np.arctan2(np.tan(delta_t)*LR,(LF+LR))

        # dt = t_list[i+1]-t_list[i]
        dx = v_t * np.cos(yaw_t + beta_t) * dt
        dy = v_t * np.sin(yaw_t + beta_t) * dt
        
        x += dx
        y += dy
        x_list.append(x)
        y_list.append(y)
        delta_list.append(delta_t)
        
    print(len(p_list[st:ed]))
    # t = t_20hz[:-1]
    plt.scatter(x_list,y_list, c='green',s=2,marker='o', label='predict')
    # plt.xlim([-60,0])
    plt.scatter(p_smooth[st:ed,0],p_smooth[st:ed,1], c='red', alpha=0.5, s=2, label='ground truth')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title("Open Loop Trajectory (Vicon input)\n"+log_dir+"\n"+f"{vel_type}\n"+f"{steer_type}")
    plt.axis('equal')
    plt.legend()

In [105]:
f = lambda steer_type='real steer', vel_type='real vel', real_yaw=False, st=0, ed=-106, frequency=16.6, speed_max=5.7, \
            delta_offset=-0.11, beta_offset=0.0, delta_proj=0.33,delay=4: plot_open_loop_trajectory(obs_np[:,:4], obs_np[:,2], action_list, LF, LR, delta_proj=delta_proj,
                                                speed_max=speed_max, delta_offset = delta_offset, frequency=frequency,
                                                steer_type=steer_type, vel_type=vel_type, real_yaw=real_yaw, st=st,ed=ed,beta_offset=beta_offset,delay=delay)

interact(f, 
                     real_yaw=[True,False],
                     vel_type=['real vel',  'target vel', 'obs vel'],
                    steer_type = ['real steer', 'target steer'],
                     frequency=(1, 100, .1),
                    speed_max=(4., 8., 0.1), 
                    steer_max=(0.2, 0.8, 0.01),
                    delta_offset=(-np.pi, np.pi, 0.01),
                     delta_proj=(-2., 2, 0.01),
                    st=(0, 2000, 1),
                     ed=(-1000, -1, 1),
                     beta_offset = (-np.pi,np.pi,0.01),
                     delay=(0,10,1),
        )

interactive(children=(Dropdown(description='steer_type', options=('real steer', 'target steer'), value='real s…

<function __main__.<lambda>(steer_type='real steer', vel_type='real vel', real_yaw=False, st=0, ed=-106, frequency=16.6, speed_max=5.7, delta_offset=-0.11, beta_offset=0.0, delta_proj=0.33, delay=4)>

## Compare Smoothed Vel, Observed Vel, and Deducted Vel

In [94]:
def plot_open_loop_trajectory(p_list, yaw_list, LF, LR, st=0, ed=200, frequency=16.6,):
    dt = 1./frequency

    obs_vel_list = p_list[:, 3]
    obs_vel_list[obs_vel_list < .0] = .0
    p_list = p_list[:, :2]
    p_smooth, vel, vel_vec = pos2vel_savgol(p_list,window_length=5, delta=dt)
    vel_calc = np.linalg.norm(p_list[1:,:2] - p_list[:-1,:2], axis=-1) /dt
    # delta, beta = calc_delta_v(vel_vec, yaw_list, LF, LR)
   
    plt.plot(obs_vel_list[st:ed], label='obs vel')
    plt.plot(vel[st:ed], label='smoothed vel')
    # plt.plot(vel_calc[st-1:ed-1], label='deducted vel')
    # print(vel_calc)
    plt.legend()

In [97]:
f = lambda st=1, ed=100, frequency=16.6: plot_open_loop_trajectory(obs_np[:,:4], obs_np[:,2], LF, LR, st=st,ed=ed,frequency=frequency,)

interact(f, 
         st=(0,len(obs_np),1),
        ed=(0,len(obs_np),1),
         frequency=(1, 20, .1),
)

interactive(children=(IntSlider(value=1, description='st', max=210), IntSlider(value=100, description='ed', ma…

<function __main__.<lambda>(st=1, ed=100, frequency=16.6)>

In [64]:
obs_np.shape

(210, 7)